In [1]:
import argparse
import torchvision
from torchvision import transforms
import torch.utils.data as data
import torch
import numpy as np
import torch.nn.functional as F
from utils.Forward_model import Holo_Generator


device='cuda:0'
parser = argparse.ArgumentParser()
args = parser.parse_args()

train_holo_list_style = [0.2]
train_holo_list_content = [round(float(i), 3) for i in np.arange(0.4, 0.9, 0.1)]
args.distance_normalize = 1.0
args.distance_normalize_constant = 0
args.pixel_size = 1.5e-6
args.phase_normalize = 1
args.wavelength=532e-9
model_forward = Holo_Generator(args).to(device)

transform_img = transforms.Compose([transforms.Resize([64, 64]), transforms.Grayscale(), transforms.ToTensor()])
dataset = torchvision.datasets.MNIST(root='/mnt/mooo/CS/style transfer based holographic imaging/data', download=True, train=True, transform=transform_img)
data = iter(data.DataLoader(dataset, batch_size=1, shuffle=False))

transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
                                    transforms.RandomAffine(degrees=0, translate=[0.1, 0.1], fill=0.0)])

for _ in range(20):
    style_origin = next(data)[0].to(device)



for idx in range(100):
    # for _ in range(4):
    content_images = transforms.functional.rotate(style_origin, angle =90/(idx+1))
    content_images = transforms.functional.affine(content_images, translate=[-0.1+ 0.2*idx/100, -0.1+ 0.2*idx/100])
    distance_style = torch.Tensor([train_holo_list_style[np.random.randint(low=0, high=len(train_holo_list_style),
                                                                        size=1)[0]] for _ in range(1)]).reshape([-1, 1, 1, 1]).to(device).float()
    distance_style = -args.distance_normalize_constant + distance_style/args.distance_normalize
    distance_content = torch.Tensor([train_holo_list_style[0] + (train_holo_list_style[-1]-train_holo_list_style[0])*idx/100]).reshape([-1, 1, 1, 1]).to(device).float()
    distance_content = -args.distance_normalize_constant + distance_style/args.distance_normalize


    phase_style = (F.pad(style_origin, (32, 32, 32, 32), 'constant', 0)).to(device).float()
    amplitude = torch.ones_like(phase_style).to(device).float() * 0.6
    phase_content_origin = (F.pad(content_images, (32, 32, 32, 32), 'constant', 0)).to(device).float()
    phase_style = transform(phase_style)
    style_holo = model_forward(amplitude, phase_style, distance_style).to(device).float().detach()
    # save_holo = style_holo
    phase_content = transform(phase_content_origin)
    content_holo = model_forward(amplitude, phase_content, distance_content).to(device).float().detach()
    
    # distance_content = torch.Tensor([d_sample]).reshape([-1, 1, 1, 1]).to(device).float()
    # distance_content = -args.distance_normalize_constant + distance_content/args.distance_normalize
    
    if idx%10 == 0:
        plt.imshow(content_holo.cpu().detach().numpy()[0][0])
        plt.show()
    

        
    # torch.save(style_holo, 'test_cs/test_style_holo_%d.pt'%idx)
    # torch.save(content_holo, 'test_cs/test_content_holo_%d.pt'%idx)
    # torch.save(distance_style, 'test_cs/test_distance_style_%d.pt'%idx)
    # torch.save(distance_content, 'test_cs/test_distance_content_%d.pt'%idx)
    # torch.save(amplitude, 'test_cs/test_gt_amplitude_%d.pt'%idx)
    # torch.save(phase_content, 'test_cs/test_gt_phase_%d.pt'%idx)



RuntimeError: operator torchvision::nms does not exist